Import necessary libraries and packages.

In [2]:
import numpy as np
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image

print('Imports complete.')

Imports complete.


Create funtion to read images into arrays of image data and label (*i.e  non, very mildly, mildly, or moderately demented*)

In [3]:
# we're getting the set type as a string and stages as a list of strings for each dir name
def cv2_read_path_train(set_type, stages):
    ''' Processes images given the file path and naming conventions'''
    
    # list to hold image data - converted to numpy array on return
    image_array = []
    
    # list to hold labels - converted to numpy array on return
    label_array = []
    
    # Label encoding dictionary
    name_dict = {'NonDemented' : 0,
                 'VeryMildDemented' : 1,
                 'MildDemented' : 2,
                 'ModerateDemented' : 3}
    
    # Iterate through stages
    for stage in stages:
        # Iterate through every image in specified directory
        for filename in glob.glob(f'Alzheimers_Dataset/{set_type}/{stage}/*.jpg'):
            # Read image
            IMG_SIZE = 100
            img = cv2.imread(filename, cv2.COLOR_BGR2RGB)
            new_array = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            # Add image to list, SCALED
            image_array.append(new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)/255)
            # Add label
            label_array.append(name_dict[stage])

    return np.array(image_array)[:,0,:,:,:], np.array(label_array) 

Now I'm creating the list of stages, then using the function I created to get the training data.

In [4]:
# Name of dir for each stage
all_stages = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']

# Create trainng sets
X_train, y_train = cv2_read_path_train(set_type='train', stages=all_stages)

IndexError: too many indices for array: array is 1-dimensional, but 5 were indexed